<a href="https://colab.research.google.com/github/jauber6/jauber6/blob/main/Software_engineer_(_Part_2_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install networkx

In [ ]:
# inventory_system.py
import time
from collections import deque
import heapq
import networkx as nx
from typing import Dict, List, Tuple

# Hash table for inventory: {product_id: {"name": str, "quantity": int, "volume": float, "turnover": float}}
inventory: Dict[str, Dict[str, any]] = {}

# FIFO queue for order processing
order_queue = deque()

# Priority queue for space optimization (based on turnover rate)
priority_queue: List[Tuple[float, str]] = []

# Binary Search Tree for sorted reporting
class BSTNode:
    def __init__(self, product_id, quantity):
        self.product_id = product_id
        self.quantity = quantity
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None

    def insert(self, product_id: str, quantity: int):
        self.root = self._insert(self.root, product_id, quantity)

    def _insert(self, node, product_id, quantity):
        if not node:
            return BSTNode(product_id, quantity)
        if product_id < node.product_id:
            node.left = self._insert(node.left, product_id, quantity)
        else:
            node.right = self._insert(node.right, product_id, quantity)
        return node

    def sorted_report(self):
        result = []
        self._inorder(self.root, result)
        return result

    def _inorder(self, node, result):
        if node:
            self._inorder(node.left, result)
            result.append({"product_id": node.product_id, "quantity": node.quantity})
            self._inorder(node.right, result)

bst = BST()

# Graph for warehouse network
warehouse_graph = nx.Graph()

# Warehouse capacity in cubic meters
WAREHOUSE_CAPACITY = 10000.0
current_volume = 0.0

def add_product(product_id: str, name: str, quantity: int, volume: float, turnover: float) -> bool:
    """Add or update a product in the inventory."""
    try:
        global current_volume
        if quantity < 0 or volume < 0 or turnover < 0:
            raise ValueError("Quantity, volume, and turnover must be non-negative.")
        total_volume = quantity * volume
        if current_volume + total_volume > WAREHOUSE_CAPACITY:
            raise ValueError("Insufficient warehouse space.")
        inventory[product_id] = {"name": name, "quantity": quantity, "volume": volume, "turnover": turnover}
        current_volume += total_volume
        bst.insert(product_id, quantity)
        heapq.heappush(priority_queue, (-turnover, product_id))  # Negative for max-heap
        return True
    except ValueError as e:
        print(f"Error adding product: {e}")
        return False

def process_order(product_id: str, quantity: int, order_type: str) -> bool:
    """Process incoming or outgoing orders using FIFO queue."""
    try:
        if product_id not in inventory:
            raise ValueError("Product not found.")
        if quantity < 0:
            raise ValueError("Quantity must be non-negative.")
        if order_type == "outgoing" and inventory[product_id]["quantity"] < quantity:
            raise ValueError("Insufficient stock for outgoing order.")

        global current_volume
        if order_type == "incoming":
            inventory[product_id]["quantity"] += quantity
            current_volume += quantity * inventory[product_id]["volume"]
        elif order_type == "outgoing":
            inventory[product_id]["quantity"] -= quantity
            current_volume -= quantity * inventory[product_id]["volume"]
        else:
            raise ValueError("Invalid order type. Use 'incoming' or 'outgoing'.")

        order_queue.append({"order_id": f"ORD{len(order_queue) + 1}", "product_id": product_id,
                           "quantity": quantity, "type": order_type, "timestamp": time.time()})
        bst.insert(product_id, inventory[product_id]["quantity"])
        return True
    except ValueError as e:
        print(f"Error processing order: {e}")
        return False

def optimize_space() -> Tuple[float, List[str]]:
    """Calculate space usage and suggest high-turnover products for priority storage."""
    try:
        used_volume = sum(item["quantity"] * item["volume"] for item in inventory.values())
        usage_percentage = (used_volume / WAREHOUSE_CAPACITY) * 100
        suggestions = []
        if usage_percentage > 80:
            # Suggest prioritizing high-turnover products
            top_products = heapq.nsmallest(3, priority_queue)  # Get top 3 by turnover
            suggestions = [f"Prioritize {inventory[pid]['name']} (ID: {pid})" for _, pid in top_products]
        return usage_percentage, suggestions
    except Exception as e:
        print(f"Error in space optimization: {e}")
        return 0.0, []

def generate_report() -> List[Dict[str, any]]:
    """Generate sorted inventory report using BST."""
    try:
        return bst.sorted_report()
    except Exception as e:
        print(f"Error generating report: {e}")
        return []

def route_order(start_warehouse: str, end_warehouse: str) -> List[str]:
    """Find optimal shipping route between warehouses using graph."""
    try:
        if not warehouse_graph.has_node(start_warehouse) or not warehouse_graph.has_node(end_warehouse):
            raise ValueError("Invalid warehouse ID.")
        return nx.shortest_path(warehouse_graph, start_warehouse, end_warehouse)
    except nx.NetworkXNoPath:
        print(f"No path between {start_warehouse} and {end_warehouse}")
        return []
    except ValueError as e:
        print(f"Error routing order: {e}")
        return []

In [ ]:
# test_inventory_system.py
import networkx as nx

def run_tests():
    print("Running tests...")

    # Initialize warehouse graph
    warehouse_graph.add_nodes_from(["WH1", "WH2", "WH3"])
    warehouse_graph.add_edges_from([("WH1", "WH2"), ("WH2", "WH3")])

    # Test 1: Add product (normal case)
    print("\nTest 1: Adding product P001")
    result = add_product("P001", "Laptop", 100, 0.5, 10.0)
    print(f"Add product P001: {'Success' if result else 'Failed'}")

    # Test 2: Add product with negative quantity (error case)
    print("\nTest 2: Adding product with negative quantity")
    result = add_product("P002", "Phone", -10, 0.2, 5.0)
    print(f"Add product P002: {'Success' if result else 'Failed'}")

    # Test 3: Process order (incoming)
    print("\nTest 3: Incoming order for P001")
    result = process_order("P001", 50, "incoming")
    print(f"Process order P001: {'Success' if result else 'Failed'}")

    # Test 4: Process order (outgoing, insufficient stock)
    print("\nTest 4: Outgoing order exceeding stock")
    result = process_order("P001", 200, "outgoing")
    print(f"Process order P001: {'Success' if result else 'Failed'}")

    # Test 5: Space optimization
    print("\nTest 5: Space optimization")
    usage, suggestions = optimize_space()
    print(f"Warehouse usage: {usage:.2f}%")
    print(f"Suggestions: {suggestions}")

    # Test 6: Generate report
    print("\nTest 6: Generating report")
    report = generate_report()
    print(f"Report: {report}")

    # Test 7: Route order (valid path)
    print("\nTest 7: Routing from WH1 to WH3")
    path = route_order("WH1", "WH3")
    print(f"Path: {path}")

    # Test 8: Route order (invalid warehouse)
    print("\nTest 8: Routing with invalid warehouse")
    path = route_order("WH1", "WH4")
    print(f"Path: {path}")

if __name__ == "__main__":
    run_tests()

Running tests...

Test 1: Adding product P001
Add product P001: Success

Test 2: Adding product with negative quantity
Error adding product: Quantity, volume, and turnover must be non-negative.
Add product P002: Failed

Test 3: Incoming order for P001
Process order P001: Success

Test 4: Outgoing order exceeding stock
Error processing order: Insufficient stock for outgoing order.
Process order P001: Failed

Test 5: Space optimization
Warehouse usage: 0.75%
Suggestions: []

Test 6: Generating report
Report: [{'product_id': 'P001', 'quantity': 100}, {'product_id': 'P001', 'quantity': 150}]

Test 7: Routing from WH1 to WH3
Path: ['WH1', 'WH2', 'WH3']

Test 8: Routing with invalid warehouse
Error routing order: Invalid warehouse ID.
Path: []
